# Introducción a Strands Agents

> Recuerda lanzar  el ```pip3 install -r requirements.txt``` para instalar ```strands-agents``` y el resto de nuevas dependencias

### Vamos a crear nuestro primer Agente!

In [ ]:
from strands import Agent
from strands_tools import calculator

# Initialize the agent with tools, model, and configuration
agent = Agent(
    tools=[calculator],
    system_prompt="You are a helpful assistant."
)

# Process user input
result = agent("Cuánto es 25 * 48?")

### Ejemplo: Tool para acceder a información pública en internet

In [ ]:
from strands import Agent
from strands_tools import http_request

# Researcher Agent with web capabilities
researcher_agent = Agent(
    system_prompt=(
        "Eres un analista de costes de vida."
        "1. Orienta al usuario sobre precios de productos de la canasta básica."
        "2. Consulta máximo una fuente. Es decir, en cuanto tengas un resultado, devuelve este dato al usuario."
        "3. Si no estás seguro, responde al usuario que no has podido confirmar la información solicitada"
    ),
    callback_handler=None,
    tools=[http_request]
)

researcher_agent("¿Cuál es el precio promedio de un café en una cafetería en CDMX, México?")

### Elección de Modelos. Empecemos con Nova Pro

In [ ]:
import boto3

region = boto3.session.Session().region_name

NOVA_PRO_MODEL_ID = "us.amazon.nova-pro-v1:0"
if region.startswith("eu"):
    NOVA_PRO_MODEL_ID = "eu.amazon.nova-pro-v1:0"
elif region.startswith("ap"):
    NOVA_PRO_MODEL_ID = "apac.amazon.nova-pro-v1:0"

print(f"Nova Pro Model ID: {NOVA_PRO_MODEL_ID}")

### Vamos a crear nuestro primer Agente, con Nova Pro

In [ ]:
from strands import Agent, tool
from strands.models import BedrockModel
from strands_tools import calculator

# Create your first agent
agent = Agent(
    model=BedrockModel(model_id=NOVA_PRO_MODEL_ID),
    system_prompt="You are a helpful assistant that provides concise responses.",
    tools=[calculator],
)

agent("¿Cuál es el área de un círculo con un radio de 8.26cm?")

### Structured Output

In [ ]:
from pydantic import BaseModel, Field
from strands import Agent

# 1) Define the Pydantic model
class PersonInfo(BaseModel):
    """Model that contains information about a Person"""
    name: str = Field(description="Name of the person")
    age: int = Field(description="Age of the person")
    occupation: str = Field(description="Occupation of the person")

# 2) Pass the model to the agent
agent = Agent()
result = agent(
    "Carlos Contreras is a 40 year-old prototyping architect at AWS",
    structured_output_model=PersonInfo
)

# 3) Access the `structured_output` from the result
person_info: PersonInfo = result.structured_output
print(f"Name: {person_info.name}")      # "John Smith"
print(f"Age: {person_info.age}")        # 30
print(f"Job: {person_info.occupation}") # "software engineer"

## Multimodal: Generemos algunas imágenes

In [ ]:
from strands import Agent, tool
from strands_tools import generate_image

# Artist agent that generates images based on prompts
artist = Agent(tools=[generate_image],system_prompt=(
    "You will be instructed to generate a number of images of a given subject." 
    "Vary the prompt for each generated image to create a variety of options."
    "Your final output must contain ONLY a comma-separated list of the filesystem paths of generated images."
))

artist("Genera dos imágenes de un coche de Formula 1 durante una carrera")

## Custom Tools

In [ ]:
from strands import Agent, tool

@tool
def get_user_location() -> str:
    """Get the user's location."""

    # Implement user location lookup logic here
    return "Seattle, USA"


@tool
def weather(location: str) -> str:
    """Get weather information for a location.

    Args:
        location: City or location name
    """

    # Implement weather lookup logic here
    return f"Weather for {location}: Sunny, 72°F"


def basic_example():
    agent = Agent(tools=[get_user_location, weather])
    agent("What is the weather like in my location?")


def main():
    basic_example()


if __name__ == "__main__":
    main()